In [1]:
from data.data import get_data_parquet, save_data_parquet
from conf.conf import logging,settings
from data.update_data import get_rating
import pandas as pd
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import shap
import numpy as np
import pandas as pd
import datetime as dt

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from lightfm.data import Dataset
from lightfm import LightFM

from catboost import CatBoostClassifier

from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.float_format', lambda x: '%.3f' % x)

/Users/suanow/opt/anaconda3/lib/python3.9/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
interactions = get_data_parquet('data/interactions.parquet')
print(interactions)

INFO:root:Extracting DataFrame
INFO:root:DataFrame is extracted


         year  month  day    user_id                              movie_id  \
0        2022     10    1      58073  a6889772-f1f4-45bc-9663-85b46fc8499d   
1        2022     10    1      63698  c829f262-ddf1-46b7-a896-a7efc205028c   
2        2022     10    1      66655  91f9d892-a508-4962-91e9-abacd93e0830   
3        2022     10    1      66655  58b805e1-a65c-49f4-b302-865cacaaed8a   
4        2022     10    1      67981  dbc44c07-46e2-4fb0-b57c-8b5172421683   
...       ...    ...  ...        ...                                   ...   
1799995  2022     12   30  211250020  35251c71-6198-4016-9135-6e30746667bd   
1799996  2022     12   30  211250020  70503f6b-b4a3-4241-9175-f030537083cd   
1799997  2022     12   30  211250020  b2d334f4-fd62-4acf-a4d7-56716dfe6cea   
1799998  2022     12   30  211250174  9db5e3ea-2b9c-4258-8288-79622e87626a   
1799999  2022     12   30  211250174  fd8ce585-39b2-457d-8779-6d19f2d337a5   

         watch_duration_minutes  
0                         4.8

In [5]:
movies_md = get_data_parquet(settings.PATH.movies_md)
print(movies_md)

TOMLDecodeError: Invalid value (at line 8, column 7)

In [43]:
interactions = get_rating(movies_md,interactions)

INFO:root:Calculating how long a user wacthed a certain film
INFO:root:Calculation is done


In [44]:
interactions

,year,month,day,user_id,movie_id,watch_duration_minutes,rating
0,2022,10,1,58073,a6889772-f1f4-45bc-9663-85b46fc8499d,4.800000,0.522748
1,2022,10,1,63698,c829f262-ddf1-46b7-a896-a7efc205028c,15.000000,0.104167
2,2022,10,1,66655,91f9d892-a508-4962-91e9-abacd93e0830,51.916668,0.522748
3,2022,10,1,66655,58b805e1-a65c-49f4-b302-865cacaaed8a,22.316668,0.522748
4,2022,10,1,67981,dbc44c07-46e2-4fb0-b57c-8b5172421683,117.000000,1.000000
...,...,...,...,...,...,...,...
1799995,2022,12,30,211250020,35251c71-6198-4016-9135-6e30746667bd,20.366667,0.522748
1799996,2022,12,30,211250020,70503f6b-b4a3-4241-9175-f030537083cd,85.166664,0.906028
1799997,2022,12,30,211250020,b2d334f4-fd62-4acf-a4d7-56716dfe6cea,9.666667,0.095710
1799998,2022,12,30,211250174,9db5e3ea-2b9c-4258-8288-79622e87626a,22.516666,0.147168


In [45]:
interactions_filtered = interactions

In [48]:
interactions_filtered['day'] = [str(x) for x in interactions_filtered['day']]
interactions_filtered['month'] = [str(x) for x in interactions_filtered['month']]
interactions_filtered['year'] = [str(x) for x in interactions_filtered['year']]

In [49]:
interactions_filtered['last_watch_dt']  = interactions_filtered['day'] + '-' + interactions_filtered['month'] + '-' + interactions_filtered['year']
interactions_filtered['last_watch_dt'] = pd.to_datetime(interactions_filtered['last_watch_dt'])

/Users/kirillgorbunov/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '13-10-2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
/Users/kirillgorbunov/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '14-10-2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
/Users/kirillgorbunov/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '15-10-2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
/Users/kirillgorbunov/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:10

In [51]:
# set dates params for filter
MAX_DATE = interactions_filtered['last_watch_dt'].max()
MIN_DATE = interactions_filtered['last_watch_dt'].min()
TEST_INTERVAL_DAYS = 14
TEST_MAX_DATE = MAX_DATE - dt.timedelta(days = TEST_INTERVAL_DAYS)

print(f"min date in filtered interactions: {MAX_DATE}")
print(f"max date in filtered interactions:: {MIN_DATE}")
print(f"test max date to split:: {TEST_MAX_DATE}")

min date in filtered interactions: 2022-12-30 00:00:00
max date in filtered interactions:: 2022-01-10 00:00:00
test max date to split:: 2022-12-16 00:00:00


In [52]:
global_train = interactions_filtered.loc[interactions_filtered['last_watch_dt'] < TEST_MAX_DATE]
global_test = interactions_filtered.loc[interactions_filtered['last_watch_dt'] >= TEST_MAX_DATE]

global_train = global_train.dropna().reset_index(drop = True)
print(global_train.shape, global_test.shape)

(1500000, 8) (300000, 8)


In [53]:
local_train_thresh = global_train['last_watch_dt'].quantile(q = .7, interpolation = 'nearest')

print(local_train_thresh)

2022-11-14 00:00:00


In [54]:
local_train = global_train.loc[global_train['last_watch_dt'] < local_train_thresh]
local_test = global_train.loc[global_train['last_watch_dt'] >= local_train_thresh]

print(local_train.shape, local_test.shape)

(1040000, 8) (460000, 8)


In [55]:
local_test = local_test.loc[local_test['user_id'].isin(local_train['user_id'].unique())]
print(local_test.shape)

(427685, 8)


In [59]:
interactions

,year,month,day,user_id,movie_id,watch_duration_minutes,rating,last_watch_dt
0,2022,10,1,58073,a6889772-f1f4-45bc-9663-85b46fc8499d,4.800,0.523,2022-01-10
1,2022,10,1,63698,c829f262-ddf1-46b7-a896-a7efc205028c,15.000,0.104,2022-01-10
2,2022,10,1,66655,91f9d892-a508-4962-91e9-abacd93e0830,51.917,0.523,2022-01-10
3,2022,10,1,66655,58b805e1-a65c-49f4-b302-865cacaaed8a,22.317,0.523,2022-01-10
4,2022,10,1,67981,dbc44c07-46e2-4fb0-b57c-8b5172421683,117.000,1.000,2022-01-10
...,...,...,...,...,...,...,...,...
1799995,2022,12,30,211250020,35251c71-6198-4016-9135-6e30746667bd,20.367,0.523,2022-12-30
1799996,2022,12,30,211250020,70503f6b-b4a3-4241-9175-f030537083cd,85.167,0.906,2022-12-30
1799997,2022,12,30,211250020,b2d334f4-fd62-4acf-a4d7-56716dfe6cea,9.667,0.096,2022-12-30
1799998,2022,12,30,211250174,9db5e3ea-2b9c-4258-8288-79622e87626a,22.517,0.147,2022-12-30


In [60]:
# init class
dataset = Dataset()

# fit tuple of user and movie interactions
dataset.fit(local_train['user_id'].unique(), local_train['movie_id'].unique())

In [61]:
lightfm_mapping = dataset.mapping()
lightfm_mapping = {
    'users_mapping': lightfm_mapping[0],
    'user_features_mapping': lightfm_mapping[1],
    'items_mapping': lightfm_mapping[2],
    'item_features_mapping': lightfm_mapping[3],
}
print('user mapper length - ', len(lightfm_mapping['users_mapping']))
print('user features mapper length - ', len(lightfm_mapping['user_features_mapping']))
print('movies mapper length - ', len(lightfm_mapping['items_mapping']))
print('Users movie features mapper length - ', len(lightfm_mapping['item_features_mapping']))

user mapper length -  104948
user features mapper length -  104948
movies mapper length -  16924
Users movie features mapper length -  16924


In [67]:
lightfm_mapping['users_inv_mapping'] = {v: k for k, v in lightfm_mapping['users_mapping'].items()}
lightfm_mapping['items_inv_mapping'] = {v: k for k, v in lightfm_mapping['items_mapping'].items()}

In [70]:
item_name_mapper = dict(zip(movies_md['movie_id'], movies_md['title']))

In [71]:
def df_to_tuple_iterator(df: pd.DataFrame):
    '''
    :df: pd.DataFrame, interactions dataframe
    returns iterator
    '''
    return zip(*df.values.T)

In [73]:
train_mat, train_mat_weights = dataset.build_interactions(df_to_tuple_iterator(local_train[['user_id', 'movie_id']]))

In [74]:
train_mat

<104948x16924 sparse matrix of type '<class 'numpy.int32'>'
	with 1040000 stored elements in COOrdinate format>

In [75]:
train_mat_weights

<104948x16924 sparse matrix of type '<class 'numpy.float32'>'
	with 1040000 stored elements in COOrdinate format>

In [76]:
# set params
NO_COMPONENTS = 64
LEARNING_RATE = .03
LOSS = 'warp'
MAX_SAMPLED = 5
RANDOM_STATE = 42
EPOCHS = 20

In [77]:
lfm_model = LightFM(
    no_components = NO_COMPONENTS,
    learning_rate = LEARNING_RATE,
    loss = LOSS,
    max_sampled = MAX_SAMPLED,
    random_state = RANDOM_STATE
    )

In [78]:
for _ in tqdm(range(EPOCHS), total = EPOCHS):
    lfm_model.fit_partial(
        train_mat,
        num_threads = 4
    )

100%|████████████████████████████████████████████████████| 20/20 [00:23<00:00,  1.18s/it]


In [79]:
top_N = 10
user_id = local_train['user_id'][100]
row_id = lightfm_mapping['users_mapping'][user_id]
print(f'Rekko for user {user_id}, row number in matrix - {row_id}')

Rekko for user 120463, row number in matrix - 69


In [80]:
# item indices
all_cols = list(lightfm_mapping['items_mapping'].values())
len(all_cols)

# predictions
pred = lfm_model.predict(
    row_id,
    all_cols,
    num_threads = 4)
pred, pred.shape

# sort and final postprocessing
top_cols = np.argpartition(pred, -np.arange(top_N))[-top_N:][::-1]
top_cols

array([  81,   82,  498, 2473,   83,  170,    2, 2679, 2426, 3971])

In [81]:
# pandas dataframe for convenience
recs = pd.DataFrame({'col_id': top_cols})
recs['item_id'] = recs['col_id'].map(lightfm_mapping['items_inv_mapping'].get)
recs['title'] = recs['item_id'].map(item_name_mapper)
recs

,col_id,item_id,title
0,81,a3bc2312-4335-4fbd-93bb-3d5f1496ec89,Чёрный ястреб
1,82,75d004d8-78fc-4462-9d03-69a20d023c55,Убить Билла 2
2,498,8ebe387b-dd82-4090-832e-e5248e0950cd,Большой куш
3,2473,debe0e17-7956-431a-b9bb-75cb583327f6,Хроники Риддика
4,83,42effaef-f937-42a2-a902-a4bf6f512c9d,Гладиатор
5,170,994fb73d-a7d8-43fc-be42-0aa451496de5,Бесславные ублюдки
6,2,91f9d892-a508-4962-91e9-abacd93e0830,1703
7,2679,7d7583b5-bb98-41a9-8ace-5f8b35b98620,Криминальное чтиво
8,2426,296ff345-0df5-4492-99ea-cb5dc2018b78,Обитель зла 2: Апокалипсис
9,3971,051ca6b4-35b2-476c-b59a-134b83a4e44e,Звёздный десант


In [82]:
# make predictions for all users in test
local_test_preds = pd.DataFrame({
    'user_id': local_test['user_id'].unique()
})
len(local_test_preds)

54247

In [83]:
def generate_lightfm_recs_mapper(
        model: object,
        item_ids: list,
        known_items: dict,
        user_features: list,
        item_features: list,
        N: int,
        user_mapping: dict,
        item_inv_mapping: dict,
        num_threads: int = 4
        ):
    def _recs_mapper(user):
        user_id = user_mapping[user]
        recs = model.predict(
            user_id,
            item_ids,
            user_features = user_features,
            item_features = item_features,
            num_threads = num_threads)
        
        additional_N = len(known_items[user_id]) if user_id in known_items else 0
        total_N = N + additional_N
        top_cols = np.argpartition(recs, -np.arange(total_N))[-total_N:][::-1]
        
        final_recs = [item_inv_mapping[item] for item in top_cols]
        if additional_N > 0:
            filter_items = known_items[user_id]
            final_recs = [item for item in final_recs if item not in filter_items]
        return final_recs[:N]
    return _recs_mapper


In [84]:
# init mapper to get predictions
mapper = generate_lightfm_recs_mapper(
    lfm_model, 
    item_ids = all_cols, 
    known_items = dict(),
    N = top_N,
    user_features = None, 
    item_features = None, 
    user_mapping = lightfm_mapping['users_mapping'],
    item_inv_mapping = lightfm_mapping['items_inv_mapping'],
    num_threads = 20
)

In [85]:
# get predictions
local_test_preds['movie_id'] = local_test_preds['user_id'].map(mapper)

In [88]:
local_test_preds = local_test_preds.explode('movie_id')
local_test_preds['rank'] = local_test_preds.groupby('user_id').cumcount() + 1 
local_test_preds['item_name'] = local_test_preds['movie_id'].map(item_name_mapper)
print(f'Data shape{local_test_preds.shape}')
local_test_preds.head(50)

Data shape(542470, 4)


,user_id,movie_id,rank,item_name
0,59460,c762cc9f-e347-4022-ad5e-3b464be6d5d2,1,Мстители: Финал
0,59460,0436e58d-2544-4328-a25d-f28b20b8dacf,2,Стражи Галактики
0,59460,8797890f-7d58-4afd-82af-010c5dc476a8,3,Мстители: Война бесконечности
0,59460,8cd91359-a447-411f-930a-2d14f1618d00,4,Стражи Галактики. Часть 2
0,59460,980cb546-9583-4a94-a00f-358322dc57e3,5,Шан-Чи и легенда десяти колец
0,59460,9bd6c733-decf-4e00-9586-8dd19719adc4,6,Чёрная Вдова
0,59460,beb507d5-5568-41c7-903a-c739a4c509dd,7,Капитан Марвел
0,59460,8afbe629-e665-4ae5-8be1-1a1a6382f1b6,8,Великий уравнитель 2
0,59460,0b92a93d-5869-44ea-b348-9af960727a45,9,Тор: Рагнарёк
0,59460,9992a169-0f6f-42ba-aebc-815d2fa689e0,10,Солт


In [90]:
local_test_preds.loc[local_test_preds['user_id']==66655]

,user_id,movie_id,rank,item_name
41031,66655,91f9d892-a508-4962-91e9-abacd93e0830,1,1703
41031,66655,0f33af53-5d2d-4c82-a2e9-307fcabdcb2d,2,Кунгур
41031,66655,f58a153d-6c9a-46ac-92ad-5531a40d7d17,3,Замерзшие
41031,66655,e088637d-967f-478b-8158-98b90aded146,4,Сердце Пармы
41031,66655,5a3bf8a0-3054-448b-b524-025abea0cac3,5,Аксентьев
41031,66655,955c3acd-c6d0-4ae4-89e4-3e0b067071e9,6,"Что делать женщине, если у неё два любовника, ..."
41031,66655,703b202e-1774-4b93-8a0c-e848d7d3abcc,7,Начать сначала
41031,66655,09ad6c80-43a5-4afe-aa59-d413dafca498,8,Либерея: Охотники за сокровищами
41031,66655,42f5cb31-f333-426c-a95b-8e106741de38,9,Вышка
41031,66655,4248fac7-aaf7-4af0-8b2f-031075390cbc,10,Ирония судьбы в Голливуде


In [92]:
# sense check for diversity of recommendations
local_test_preds.movie_id.nunique()

2767

In [94]:
positive_preds = pd.merge(local_test_preds, local_test, how = 'inner', on = ['user_id', 'movie_id'])
positive_preds['target'] = 1
positive_preds.shape

(39267, 11)

In [96]:
interactions

,year,month,day,user_id,movie_id,watch_duration_minutes,rating,last_watch_dt
0,2022,10,1,58073,a6889772-f1f4-45bc-9663-85b46fc8499d,4.800,0.523,2022-01-10
1,2022,10,1,63698,c829f262-ddf1-46b7-a896-a7efc205028c,15.000,0.104,2022-01-10
2,2022,10,1,66655,91f9d892-a508-4962-91e9-abacd93e0830,51.917,0.523,2022-01-10
3,2022,10,1,66655,58b805e1-a65c-49f4-b302-865cacaaed8a,22.317,0.523,2022-01-10
4,2022,10,1,67981,dbc44c07-46e2-4fb0-b57c-8b5172421683,117.000,1.000,2022-01-10
...,...,...,...,...,...,...,...,...
1799995,2022,12,30,211250020,35251c71-6198-4016-9135-6e30746667bd,20.367,0.523,2022-12-30
1799996,2022,12,30,211250020,70503f6b-b4a3-4241-9175-f030537083cd,85.167,0.906,2022-12-30
1799997,2022,12,30,211250020,b2d334f4-fd62-4acf-a4d7-56716dfe6cea,9.667,0.096,2022-12-30
1799998,2022,12,30,211250174,9db5e3ea-2b9c-4258-8288-79622e87626a,22.517,0.147,2022-12-30


In [97]:
negative_preds = pd.merge(local_test_preds, local_test, how = 'left', on = ['user_id', 'movie_id'])
negative_preds = negative_preds.loc[negative_preds['rating'].isnull()].sample(frac = .2)
negative_preds['target'] = 0
negative_preds.shape


(102931, 11)

In [98]:
train_users, test_users = train_test_split(
    local_test['user_id'].unique(),
    test_size = .2,
    random_state = 13
    )

In [99]:
cbm_train_set = shuffle(
    pd.concat(
    [positive_preds.loc[positive_preds['user_id'].isin(train_users)],
    negative_preds.loc[negative_preds['user_id'].isin(train_users)]]
    )
)

In [100]:
cbm_test_set = shuffle(
    pd.concat(
    [positive_preds.loc[positive_preds['user_id'].isin(test_users)],
    negative_preds.loc[negative_preds['user_id'].isin(test_users)]]
    )
)

In [101]:
print(f'TRAIN: {cbm_train_set.describe()} \n, TEST: {cbm_test_set.describe()}')

TRAIN:             user_id       rank  watch_duration_minutes    rating     target
count    113663.000 113663.000               31426.000 31426.000 113663.000
mean  155371796.280      5.251                  49.821     0.524      0.276
std    97606372.793      2.913                  47.070     0.303      0.447
min       56250.000      1.000                   0.017     0.000      0.000
25%    23965258.000      3.000                  11.867     0.304      0.000
50%   211313489.000      5.000                  42.633     0.523      0.000
75%   228427845.000      8.000                  78.800     0.778      1.000
max   239449357.000     10.000                 983.417     1.000      1.000 
, TEST:             user_id      rank  watch_duration_minutes   rating    target
count     28535.000 28535.000                7841.000 7841.000 28535.000
mean  156093061.245     5.264                  50.053    0.524     0.275
std    97383153.335     2.930                  47.228    0.301     0.446
min     